# PART1 
## 1. Start by creating a new Notebook for this assignment.

In [57]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#scrape table by BeautifulSoup
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv
print('libary & BeautifulSoup & csv installed')


Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

libary & BeautifulSoup & csv installed


# 2. Scrape table from Wikipedia page and create dataframe


In [60]:
#Secure Sockets Layer (SSL) errors ignored
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE
print('SSL certificate errors ignored.')


SSL certificate errors ignored.


In [62]:
# processing HTML in the Python
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
web= requests.get(url)
source= web.text
# "lxml" is the most feature-rich and easy-to-use library for processing XML and HTML in the Python
soup= BeautifulSoup(source, 'lxml')
print('soup ready')


soup ready


In [66]:
#find a table in a Wikipedia page
table = soup.find('table',{'class':'wikitable sortable'})


In [67]:
#find rows in a Wikipedia table, information get from inspect of the page
table_rows = table.find_all('tr')


In [69]:
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')]) 
    #Leading and trailing whitespaces are removed by .strip()

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows

print(df.head(12))


   PostalCode           Borough                                Neighbourhood
1         M1A      Not assigned                                 Not assigned
2         M2A      Not assigned                                 Not assigned
3         M3A        North York                                    Parkwoods
4         M4A        North York                             Victoria Village
5         M5A  Downtown Toronto                    Regent Park, Harbourfront
6         M6A        North York             Lawrence Manor, Lawrence Heights
7         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government
8         M8A      Not assigned                                 Not assigned
9         M9A         Etobicoke      Islington Avenue, Humber Valley Village
10        M1B       Scarborough                               Malvern, Rouge
11        M2B      Not assigned                                 Not assigned
12        M3B        North York                                    Don Mills

# 3. Clean dataframe

~The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.

~Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

~More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

~If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

~Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

~In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [70]:
#Ignore cells with a borough that is Not assigned
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df.head()


,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [81]:
df1 = df.reset_index(drop=True)
df1.head(12)


,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [79]:
df1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   PostalCode     103 non-null    object
 1   Borough        103 non-null    object
 2   Neighbourhood  103 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB


#~More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.


In [83]:
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x)) 
# group by postalcode, aggragate colums by joint, use "," to seperate each value after aggragating
df2 = df1.reset_index(drop=True)
df2.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


~If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [85]:
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']

df2.head(12)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


~In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [86]:
df2.shape

(103, 3)

# Part2

## Get the latitude and the longitude coordinates of each neighborhood. 

Since the given package is not reliable, i will use the csv file to create the requested dataframe


In [117]:
# get Geospatial_Coordinates.csv
Geotb=pd.read_csv('/Users/shanshanbai/Downloads/Geospatial_Coordinates.csv')
Geotb.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [118]:
Geotb.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
Geotb.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [120]:
df_geotb = pd.DataFrame(data=Geotb)

In [121]:
df_geotb.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [123]:
merged_inner = pd.merge(left=df2, right=df_geotb, left_on='PostalCode', right_on='PostalCode')
merged_inner.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [124]:
merged_inner.shape

(103, 5)

# Part3

##Explore and cluster the neighborhoods in Toronto

In [125]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [126]:
merged_inner.to_json(path_or_buf='geo_toronto.json', orient='table')

In [127]:
with open('geo_toronto.json') as json_data:
    Toronto_data = json.load(json_data)
    

In [130]:
Toronto_data

{'schema': {'fields': [{'name': 'index', 'type': 'integer'},
   {'name': 'PostalCode', 'type': 'string'},
   {'name': 'Borough', 'type': 'string'},
   {'name': 'Neighbourhood', 'type': 'string'},
   {'name': 'Latitude', 'type': 'number'},
   {'name': 'Longitude', 'type': 'number'}],
  'primaryKey': ['index'],
  'pandas_version': '0.20.0'},
 'data': [{'index': 0,
   'PostalCode': 'M3A',
   'Borough': 'North York',
   'Neighbourhood': 'Parkwoods',
   'Latitude': 43.7532586,
   'Longitude': -79.3296565},
  {'index': 1,
   'PostalCode': 'M4A',
   'Borough': 'North York',
   'Neighbourhood': 'Victoria Village',
   'Latitude': 43.7258823,
   'Longitude': -79.3155716},
  {'index': 2,
   'PostalCode': 'M5A',
   'Borough': 'Downtown Toronto',
   'Neighbourhood': 'Regent Park, Harbourfront',
   'Latitude': 43.6542599,
   'Longitude': -79.3606359},
  {'index': 3,
   'PostalCode': 'M6A',
   'Borough': 'North York',
   'Neighbourhood': 'Lawrence Manor, Lawrence Heights',
   'Latitude': 43.718518,
 

In [131]:
neighborhoods_data = Toronto_data['data']
neighborhoods_data[0]
#Let's take a look at the first item in this list.

{'index': 0,
 'PostalCode': 'M3A',
 'Borough': 'North York',
 'Neighbourhood': 'Parkwoods',
 'Latitude': 43.7532586,
 'Longitude': -79.3296565}

In [133]:
merged_inner.info()
merged_inner.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PostalCode     103 non-null    object 
 1   Borough        103 non-null    object 
 2   Neighbourhood  103 non-null    object 
 3   Latitude       103 non-null    float64
 4   Longitude      103 non-null    float64
dtypes: float64(2), object(3)
memory usage: 9.8+ KB


(103, 5)

In [134]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(merged_inner['Borough'].unique()),
        merged_inner.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


## map with folium

In [136]:
!conda install -c conda-forge folium=0.5.0 

Solving environment: done

# All requested packages already installed.



In [137]:
import pandas as pd
import folium

print('imported pandas & folium')

imported pandas & folium


In [171]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="CA_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [172]:
# create map of Toronto using latitude and longitude values
map_toronto_neighbourhoods = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(merged_inner['Latitude'], merged_inner['Longitude'], merged_inner['Borough'], merged_inner['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_neighbourhoods)  
    
map_toronto_neighbourhoods


#open map_toronto_neighbourhoods.html in browser
#if you cannot generate the maps open PGA_map_*.html from the zip file

## create a new dataframe of the Downtown Toronto data

In [185]:
downtown_data = merged_inner[merged_inner['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [186]:
address = 'Downtown Toronto, ca'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.6563221, -79.3809161.


In [188]:
#create map of Downtown Toronto using latitude and longitude values
map_Downtown = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Downtown)  
    
map_Downtown

In [189]:
CLIENT_ID = 'KEGTGNXRKWYHIGDLX1SMJ105TMAKFGDGHTIT4KMAODXIZNEE' # your Foursquare ID
CLIENT_SECRET = 'Y0LWOATB2TUF40GOTQLMUCGM12DPYPHRLMZXBYICI2BLRPOR' # your Foursquare Secret
VERSION = '20200629' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KEGTGNXRKWYHIGDLX1SMJ105TMAKFGDGHTIT4KMAODXIZNEE
CLIENT_SECRET:Y0LWOATB2TUF40GOTQLMUCGM12DPYPHRLMZXBYICI2BLRPOR


In [190]:
downtown_data.loc[0, 'Neighbourhood']

'Regent Park, Harbourfront'

In [191]:
neighborhood_latitude = downtown_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = downtown_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = downtown_data.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


In [192]:
# type your answer here
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=KEGTGNXRKWYHIGDLX1SMJ105TMAKFGDGHTIT4KMAODXIZNEE&client_secret=Y0LWOATB2TUF40GOTQLMUCGM12DPYPHRLMZXBYICI2BLRPOR&v=20200629&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [193]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ef994f24153fb4629dcb2cb'},
 'response': {'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 45,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.653446723052674,
          'lng': -79.3620167174383}],
        'distance': 143,
       

From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [194]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [195]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/Users/shanshanbai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Dominion Pub and Kitchen,Pub,43.656919,-79.358967


In [196]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

45 venues were returned by Foursquare.


## Explore Neighborhoods in Downtown Toroto

In [232]:
def getNearbyVenues(names, latitudes, longitudes, radius=800):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *downtown_venues*.

In [233]:

downtown_venues = getNearbyVenues(names=downtown_data['Neighbourhood'],
                                   latitudes=downtown_data['Latitude'],
                                   longitudes=downtown_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


#### Let's check the size of the resulting dataframe

In [234]:
print(downtown_venues.shape)
downtown_venues.head()

(1649, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Corktown Common,43.655618,-79.356211,Park
4,"Regent Park, Harbourfront",43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site


Let's check how many venues were returned for each neighborhood

In [235]:
downtown_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",30,30,30,30,30,30
Central Bay Street,100,100,100,100,100,100
Christie,63,63,63,63,63,63
Church and Wellesley,100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues

In [236]:
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 221 uniques categories.


##  Analyze Each Neighborhood

In [237]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighbourhood'] = downtown_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
owntown_onehot = downtown_onehot[fixed_columns]

owntown_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,Hobby Shop,Hostel,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Office,Optical Shop,Organic Grocery,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Track,Trail,Train Station,Tunnel,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [238]:
#let's examine the new dataframe size.
downtown_onehot.shape

(1649, 222)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [239]:
downtown_grouped = downtown_onehot.groupby('Neighbourhood').mean().reset_index()
downtown_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,Hobby Shop,Hostel,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Office,Optical Shop,Organic Grocery,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Track,Trail,Train Station,Tunnel,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.010000,0.00,0.010000,0.00,0.00,0.000000,0.00,0.000000,0.01,0.030000,0.000000,0.000000,0.00,0.01,0.01,0.00,0.03,0.000000,0.00,0.00,0.01,0.00,0.00,0.00,0.000000,0.00,0.020000,0.00,0.01,0.000000,0.000000,0.000000,0.060000,0.00,0.000000,0.000000,0.02,0.000000,0.000000,0.01,0.01,0.020000,0.100000,0.00,0.00,0.00,0.00,0.01,0.00,0.01,0.000000,0.010000,0.02,0.00,0.010000,0.01,0.000000,0.010000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.010000,0.000000,0.000000,0.01,0.000000,0.00,0.010000,0.01,0.010000,0.00,0.000000,0.00,0.00,0.020000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.01,0.01,0.020000,0.010000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.00,0.00,0.00,0.050000,0.01,0.000000,0.000000,0.00,0.020000,0.040000,0.01,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.01000

In [240]:
downtown_grouped.shape

(19, 222)

### Let's print each neighborhood along with the top 5 most common venues

In [241]:
num_top_venues = 5

for hood in downtown_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n') #enter on row between two

----Berczy Park----
                 venue  freq
0          Coffee Shop  0.10
1                 Café  0.06
2           Restaurant  0.05
3                Hotel  0.05
4  Japanese Restaurant  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0  Rental Car Location   0.1
1        Boat or Ferry   0.1
2      Harbor / Marina   0.1
3      Airport Service   0.1
4     Airport Terminal   0.1


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.08
1  Japanese Restaurant  0.05
2                 Café  0.05
3          Art Gallery  0.04
4       Clothing Store  0.04


----Christie----
               venue  freq
0  Korean Restaurant  0.11
1      Grocery Store  0.10
2        Coffee Shop  0.10
3              Diner  0.05
4               Café  0.05


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.10
1  Japanese Restaurant  0.04
2     Sushi R

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [242]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [243]:
num_top_venues1 = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues1):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = downtown_grouped['Neighbourhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues1)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Park,Beer Bar,Bakery,Cheese Shop,Creperie
1,"CN Tower, King and Spadina, Railway Lands, Har...",Boat or Ferry,Rental Car Location,Airport Service,Airport Terminal,Harbor / Marina,Sculpture Garden,Airport Lounge,Coffee Shop,Plane,Music Venue
2,Central Bay Street,Coffee Shop,Japanese Restaurant,Café,Art Gallery,Clothing Store,Park,Ramen Restaurant,Italian Restaurant,Burrito Place,Sushi Restaurant
3,Christie,Korean Restaurant,Coffee Shop,Grocery Store,Park,Diner,Café,Pizza Place,Cocktail Bar,Karaoke Bar,Indian Restaurant
4,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Men's Store,Restaurant,Park,Diner,Yoga Studio,Gay Bar,Ramen Restaurant


## Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [244]:
# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 2, 0, 4, 0, 3, 3, 0, 0, 0], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [245]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

downtown_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Restaurant,Bakery,Theater,Italian Restaurant,Café,Pub,Thai Restaurant,Breakfast Spot
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Bubble Tea Shop,Park,Bookstore,Gastropub,Italian Restaurant,Café,Sushi Restaurant,Yoga Studio,Thai Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Japanese Restaurant,Gastropub,Italian Restaurant,Ramen Restaurant,Diner,Bubble Tea Shop,Middle Eastern Restaurant,Clothing Store,Department Store
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Coffee Shop,Café,Restaurant,Gastropub,American Restaurant,Cosmetics Shop,Japanese Restaurant,Seafood Restaurant,Bakery,Department Store
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,3,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Park,Beer Bar,Bakery,Cheese Shop,Creperie


 Visualize the resulting clusters

In [246]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighbourhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##  Examine Clusters

#### Cluster 1

In [249]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Park,Restaurant,Bakery,Theater,Italian Restaurant,Café,Pub,Thai Restaurant,Breakfast Spot
1,Downtown Toronto,0,Coffee Shop,Bubble Tea Shop,Park,Bookstore,Gastropub,Italian Restaurant,Café,Sushi Restaurant,Yoga Studio,Thai Restaurant
2,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Gastropub,Italian Restaurant,Ramen Restaurant,Diner,Bubble Tea Shop,Middle Eastern Restaurant,Clothing Store,Department Store
5,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Café,Art Gallery,Clothing Store,Park,Ramen Restaurant,Italian Restaurant,Burrito Place,Sushi Restaurant
8,Downtown Toronto,0,Coffee Shop,Hotel,Park,Brewery,Japanese Restaurant,Pizza Place,Sushi Restaurant,Music Venue,Concert Hall,Café
11,Downtown Toronto,0,Café,Coffee Shop,Bakery,Restaurant,Vegetarian / Vegan Restaurant,Pizza Place,Bookstore,Italian Restaurant,Bubble Tea Shop,Thai Restaurant
12,Downtown Toronto,0,Café,Vegetarian / Vegan Restaurant,Art Gallery,Bar,Coffee Shop,Yoga Studio,Park,Bakery,Mexican Restaurant,Dessert Shop
16,Downtown Toronto,0,Coffee Shop,Pizza Place,Grocery Store,Café,Restaurant,Park,Bakery,Thai Restaurant,Pharmacy,Pub
18,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Men's Store,Restaurant,Park,Diner,Yoga Studio,Gay Bar,Ramen Restaurant


#### Cluster 2

In [250]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,1,Trail,Park,Playground,Candy Store,Grocery Store,Bank,Donut Shop,Dog Run,Doner Restaurant,Dumpling Restaurant


#### Cluster 3

In [251]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Downtown Toronto,2,Boat or Ferry,Rental Car Location,Airport Service,Airport Terminal,Harbor / Marina,Sculpture Garden,Airport Lounge,Coffee Shop,Plane,Music Venue


#### Cluster 4

In [252]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Gastropub,American Restaurant,Cosmetics Shop,Japanese Restaurant,Seafood Restaurant,Bakery,Department Store
4,Downtown Toronto,3,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Park,Beer Bar,Bakery,Cheese Shop,Creperie
7,Downtown Toronto,3,Coffee Shop,Café,Hotel,Theater,Restaurant,Gym,Art Gallery,Japanese Restaurant,Cosmetics Shop,Concert Hall
9,Downtown Toronto,3,Hotel,Coffee Shop,Café,Restaurant,Concert Hall,Seafood Restaurant,American Restaurant,Italian Restaurant,Park,Japanese Restaurant
10,Downtown Toronto,3,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Seafood Restaurant,Gym,American Restaurant,Concert Hall,Vegetarian / Vegan Restaurant
15,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Japanese Restaurant,Gastropub,Bakery,Hotel,Seafood Restaurant,Breakfast Spot,Gym
17,Downtown Toronto,3,Hotel,Coffee Shop,Café,Restaurant,Concert Hall,Japanese Restaurant,Theater,Vegetarian / Vegan Restaurant,Italian Restaurant,Beer Bar


#### Cluster 5

In [253]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,4,Korean Restaurant,Coffee Shop,Grocery Store,Park,Diner,Café,Pizza Place,Cocktail Bar,Karaoke Bar,Indian Restaurant
